In [2]:
!pip3 install dotenv 

In [ ]:
import boto3
from botocore.exceptions import ClientError
import os
from dotenv import load_dotenv

load_dotenv()

# AWS Configure
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_REGION = os.getenv('AWS_REGION', default='ap-northeast-2')

def create_dynamodb_client(region=AWS_REGION):  # 기본 리전 설정
    """DynamoDB 클라이언트를 생성합니다."""
    return boto3.client("dynamodb",
                       region_name=region,
                       aws_access_key_id=AWS_ACCESS_KEY_ID,
                       aws_secret_access_key=AWS_SECRET_ACCESS_KEY)

def execute_query(dynamodb_client, query_scan):
    """
    DynamoDB 쿼리를 실행하고 결과를 처리합니다.

    Args:
        dynamodb_client: boto3 DynamoDB 클라이언트 객체
        query_scan (dict): DynamoDB query() 메서드에 전달할 입력 파라미터
    """
    try:
        response = dynamodb_client.query(**query_scan)
        print("Query successful.")

        # 결과 처리 (예: 항목 출력)
        if 'Items' in response:
            for item in response['Items']:
                print(item)
        else:
            print("No items found.")

        return response  # 결과 반환

    except ClientError as error:
        handle_error(error)
        return None  # 또는 예외를 다시 발생(raise)
    except Exception as error: #boto3 및 연결 관련 외 에러 처리
        print(f"Unknown error while querying: {error}")
        return None

def handle_error(error):
    """
    Boto3 ClientError 예외를 처리합니다.

    Args:
        error (ClientError): 발생한 ClientError 예외
    """
    error_code = error.response['Error']['Code']
    error_message = error.response['Error']['Message']

    # 오류 코드별 도움말 문자열 (필요에 따라 추가)
    ERROR_HELP_STRINGS = {
        'ConditionalCheckFailedException': "The conditional check failed",
        'ProvisionedThroughputExceededException': "Provisioned throughput exceeded",
        'ResourceNotFoundException': "The table does not exist",
        'ValidationException': "Validation error. Check your input.",
        # 다른 오류 코드에 대한 처리 추가
    }

    error_help_string = ERROR_HELP_STRINGS.get(error_code, "An error occurred")

    print(f"[{error_code}] {error_help_string}. Error message: {error_message}")


Query successful.
{'AlbumTitle': {'S': 'Somewhat Famous'}, 'Awards': {'N': '1'}, 'Artist': {'S': 'No One You Know'}, 'SongTitle': {'S': 'Call Me Today'}}


In [ ]:
# 조회하기
def create_query_scan(table_name, artist_name, song_title):
    """
    DynamoDB 쿼리에 필요한 입력 파라미터를 생성합니다.

    Args:
        table_name (str): 테이블 이름
        artist_name (str): 아티스트 이름 (파티션 키)
        song_title (str): 노래 제목 (정렬 키)

    Returns:
        dict: DynamoDB query() 메서드에 전달할 입력 파라미터
    """
    return {
        "TableName": table_name,
        "KeyConditionExpression": "Artist = :artist_val AND SongTitle = :song_val",  # 문제 발생 부분
        # "ExpressionAttributeNames": {
        #     "#Artist": "Artist",
        #     "#Songtitle": "SongTitle"
        # },
        "ExpressionAttributeValues": {
            ":artist_val": {"S": artist_name},
            ":song_val": {"S": song_title}
        }
    }

"""메인 함수"""
# DynamoDB 클라이언트 생성
dynamodb_client = create_dynamodb_client()

# -------------------조회-----------------------

table_name = "Music"  # 테이블 이름
artist = "No One You Know"  # 찾을 아티스트
song = "Call Me Today"   # 찾을 노래 제목

query_scan = create_query_scan(table_name, artist, song)
result = execute_query(dynamodb_client, query_scan)


In [18]:
# 아이템 넣기
def create_put_item_input(table_name, item_data):
    """
    DynamoDB에 아이템을 생성(PutItem)하기 위한 입력 파라미터를 생성합니다.

    Args:
        table_name (str): 테이블 이름
        item_data (dict): 추가할 아이템 데이터 (딕셔너리 형태)
                           키는 속성 이름, 값은 {'타입': '값'} 형태

    Returns:
        dict: DynamoDB put_item() 메서드에 전달할 입력 파라미터
    """
    return {
        "TableName": table_name,
        "Item": item_data
    }


def execute_put_item(dynamodb_client, put_item_input):
    """
    DynamoDB에 아이템을 생성(PutItem)하고 결과를 처리합니다.

    Args:
        dynamodb_client: boto3 DynamoDB 클라이언트 객체
        put_item_input (dict): DynamoDB put_item() 메서드에 전달할 입력 파라미터
    """
    try:
        response = dynamodb_client.put_item(**put_item_input)
        print("Item added successfully.")
        return response

    except ClientError as error:
        handle_error(error)
        return None  # 또는 예외를 다시 발생(raise)
    except Exception as error:  # boto3 및 연결 관련 외 에러 처리
        print(f"Unknown error while adding item: {error}")
        return None

"""메인 함수"""
# DynamoDB 클라이언트 생성
dynamodb_client = create_dynamodb_client()

# -------------------입력----------------------
# 추가할 아이템 데이터 (여러 개 추가 가능)
items_to_add = [
    {
        'Artist': {'S': 'user001'},  # 파티션 키
        'SongTitle': {'S': 'John'},
        'WhatName': {'S': 'Doe'},
        'Age': {'N': '30'}  # 숫자 타입은 문자열로
    },
    {
        'Artist': {'S': 'No One Know'},
        'SongTitle': {'S': 'asdasdasdasd'},
        'AlbumTitle': {'S': 'Smith'},
        'Email': {'S': 'jane.smith@example.com'}
    },
    {
        'Artist': {'S': 'user003'},
        'SongTitle': {'S': 'Bob_J'},
        'Status': {'S': 'Active'}
    }
]

table_name = "Music"  # 테이블 이름

# 각 아이템 추가
for item_data in items_to_add:
    put_item_input = create_put_item_input(table_name, item_data)
    result = execute_put_item(dynamodb_client, put_item_input)


Item added successfully.
Item added successfully.
Item added successfully.


In [ ]:
# 아이템 삭제하기
def delete_item(dynamodb_client, table_name, key):
    """
    DynamoDB 테이블에서 단일 아이템을 삭제합니다.

    Args:
        dynamodb_client: boto3 DynamoDB 클라이언트 객체
        table_name (str): 테이블 이름
        key (dict): 삭제할 아이템의 키 (파티션 키, 정렬 키)
                   예: {'Artist': {'S': 'user001'}}
                       {'Artist': {'S': 'user002'}, 'Timestamp': {'N': '1678886400'}}
    Returns:
        dict: 삭제 결과 (성공 시), 실패 시 None
    """
    try:
        response = dynamodb_client.delete_item(
            TableName=table_name,
            Key=key
        )a
        print("Item deleted successfully.")
        return response
    except ClientError as error:
        handle_error(error)
        return None
    except Exception as error:
        print(f"Unknown error while deleting item: {error}")
        return None


def delete_item_conditional(dynamodb_client, table_name, key, condition_expression, expression_attribute_values):
    """
    조건부 삭제 (Conditional Delete)를 수행합니다.

    Args:
        dynamodb_client:  boto3 DynamoDB 클라이언트.
        table_name (str): 테이블 이름.
        key (dict): 삭제할 아이템의 키.
        condition_expression (str): 삭제 조건 표현식.
        expression_attribute_values (dict): 표현식 속성 값.

    Returns:
        dict: 삭제 결과. 실패 시 None.
    """
    try:
        response = dynamodb_client.delete_item(
            TableName=table_name,
            Key=key,
            ConditionExpression=condition_expression,
            ExpressionAttributeValues=expression_attribute_values
        )
        print("Item deleted conditionally successfully.")
        return response
    except ClientError as error:
        handle_error(error)
        return None
    except Exception as error:
        print(f"Unknown error: {error}")
        return None


def batch_delete_items(dynamodb_client, table_name, keys_to_delete):
    """
    여러 아이템을 한 번에 삭제합니다 (BatchWriteItem 사용).

    Args:
        dynamodb_client: boto3 DynamoDB 클라이언트 객체
        table_name (str): 테이블 이름
        keys_to_delete (list): 삭제할 아이템들의 키 목록 (각 키는 딕셔너리)
                           예: [{'Artist': {'S': 'user001'}}, {'Artist': {'S': 'user002'}}]

    Returns:
        dict: 삭제 결과.  실패 시 None
    """
    request_items = {
        table_name: [
            {'DeleteRequest': {'Key': key}} for key in keys_to_delete
        ]
    }

    try:
        response = dynamodb_client.batch_write_item(RequestItems=request_items)

        # 처리되지 않은 항목(UnprocessedItems)이 있는지 확인
        unprocessed_items = response.get('UnprocessedItems', {})
        if unprocessed_items:
            print("WARNING: Some items were not deleted. UnprocessedItems:", unprocessed_items)
            # TODO: UnprocessedItems 재처리 로직 추가 (필요한 경우)
            # 재시도 횟수 제한, exponential backoff 등 고려
        else:
             print("Items deleted successfully.")

        return response

    except ClientError as error:
        handle_error(error)
        return None
    except Exception as error:
        print(f"Unknown error while deleting items: {error}")
        return None

"""메인 함수"""
dynamodb_client = create_dynamodb_client()
table_name = "Music"  # 테이블 이름

# 1. 단일 아이템 삭제
key_to_delete = {'Artist': {'S': 'No One You Know'}, 'SongTitle': {'S': 'Call Me Today'}}
delete_item(dynamodb_client, table_name, key_to_delete)

# keys_to_delete = [
#     {'Artist': {'S': 'user003'}},
#     {'Artist': {'S': 'user004'}}  # 존재하지 않는 아이템도 포함 가능 (오류 발생 X)
# ]
# batch_delete_items(dynamodb_client, table_name, keys_to_delete)


Item deleted successfully.


{'ResponseMetadata': {'RequestId': 'REGJ9ETHTUBH42PKP0IF7K8A9FVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Thu, 20 Feb 2025 03:34:12 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'REGJ9ETHTUBH42PKP0IF7K8A9FVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}